In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import VotingClassifier

In [2]:
df=pd.read_csv("ENB_data.csv",sep = ',')

In [3]:
df.columns = ["Relative Compactness","Surface Area","Wall Area",
                "Roof Area", "Overall Height","Orientation","Glazing Area",
                "Glazing Area Distribution", "Heating Load", "Cooling Load"]

In [4]:
df

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load,Cooling Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28
...,...,...,...,...,...,...,...,...,...,...
763,0.64,784.0,343.0,220.50,3.5,5,0.4,5,17.88,21.40
764,0.62,808.5,367.5,220.50,3.5,2,0.4,5,16.54,16.88
765,0.62,808.5,367.5,220.50,3.5,3,0.4,5,16.44,17.11
766,0.62,808.5,367.5,220.50,3.5,4,0.4,5,16.48,16.61


In [5]:
df.corr()

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load,Cooling Load
Relative Compactness,1.000000e+00,-9.919015e-01,-2.037817e-01,-8.688234e-01,8.277473e-01,4.678592e-17,-2.960552e-15,-7.107006e-16,0.622272,0.634339
Surface Area,-9.919015e-01,1.000000e+00,1.955016e-01,8.807195e-01,-8.581477e-01,-3.459372e-17,3.636925e-15,2.438409e-15,-0.658120,-0.672999
Wall Area,-2.037817e-01,1.955016e-01,1.000000e+00,-2.923165e-01,2.809757e-01,-2.429499e-17,-8.567455e-17,2.067384e-16,0.455671,0.427117
Roof Area,-8.688234e-01,8.807195e-01,-2.923165e-01,1.000000e+00,-9.725122e-01,-5.830058e-17,-1.759011e-15,-1.078071e-15,-0.861828,-0.862547
Overall Height,8.277473e-01,-8.581477e-01,2.809757e-01,-9.725122e-01,1.000000e+00,4.492205e-17,1.489134e-17,-2.920613e-17,0.889431,0.895785
Orientation,4.678592e-17,-3.459372e-17,-2.429499e-17,-5.830058e-17,4.492205e-17,1.000000e+00,-9.406007e-16,-2.549352e-16,-0.002587,0.014290
Glazing Area,-2.960552e-15,3.636925e-15,-8.567455e-17,-1.759011e-15,1.489134e-17,-9.406007e-16,1.000000e+00,2.129642e-01,0.269841,0.207505
Glazing Area Distribution,-7.107006e-16,2.438409e-15,2.067384e-16,-1.078071e-15,-2.920613e-17,-2.549352e-16,2.129642e-01,1.000000e+00,0.087368,0.050525
Heating Load,6.222722e-01,-6.581202e-01,4.556712e-01,-8.618283e-01,8.894307e-01,-2.586534e-03,2.698410e-01,8.736759e-02,1.000000,0.975862
Cooling Load,6.343391e-01,-6.729989e-01,4.271170e-01,-8.625466e-01,8.957852e-01,1.428960e-02,2.075050e-01,5.052512e-02,0.975862,1.000000


In [6]:
#Roof Area et Overall Height sont les plus corrélées aux deux variables cibles	

In [7]:
df['total_charges']=df["Heating Load"]+df["Cooling Load"]

In [8]:
q=df['total_charges'].quantile([0.25, 0.5, 0.75]).values
def classify(x):
    if x<=q[0]:
        return 0
    if x>q[0] and x<=q[1]:
        return 1
    if x>q[1] and x<=q[2]:
        return 2
    else :
        return 3
charge_classes=df['total_charges'].apply(classify)

In [29]:
q

array([28.75 , 40.97 , 64.335])

In [28]:
charge_classes.value_counts()

1    192
2    192
3    192
0    192
Name: total_charges, dtype: int64

In [9]:
data=df.drop(['Heating Load','Cooling Load','total_charges'],axis=1)

In [10]:
X_train,X_test,y_train,y_test=train_test_split(data,charge_classes,test_size=0.2)
#PREPROCESSING
sc=preprocessing.StandardScaler()
X_train_transformed=sc.fit_transform(X_train)
X_test_transformed=sc.transform(X_test)

**K plus proches voisins**

In [11]:
parameters= { 'n_neighbors' : range(2,51) }
knn = neighbors.KNeighborsClassifier()        
grid=model_selection.GridSearchCV(knn,parameters)
grid.fit(X_train_transformed,y_train.values.ravel())

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(2, 51)})

In [12]:
grid.best_params_

{'n_neighbors': 3}

In [13]:
# Prediction on test set
Y_pred = grid.predict( X_test_transformed )

In [14]:
model_knn=grid.best_estimator_
model_knn.score(X_test_transformed,y_test)

0.8701298701298701

In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix=confusion_matrix(y_test,Y_pred)
confusion_matrix

array([[33,  1,  0,  0],
       [ 3, 28,  3,  0],
       [ 0,  3, 36,  4],
       [ 0,  1,  5, 37]], dtype=int64)

**SVM**

In [16]:
# import support vector classifier
parameters = { 'kernel' : ['rbf','linear'],'C':[0.1,1,10,50] }
model = svm.SVC()        
grid=model_selection.GridSearchCV(model,parameters)
grid.fit(X_train_transformed,y_train.values.ravel())

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 50], 'kernel': ['rbf', 'linear']})

In [17]:
grid.best_params_

{'C': 50, 'kernel': 'rbf'}

In [18]:
Y_pred = grid.predict( X_test_transformed )

In [19]:
model_sv=grid.best_estimator_
model_sv.score(X_test_transformed,y_test)

0.922077922077922

In [20]:
from sklearn.metrics import confusion_matrix
confusion_matrix=confusion_matrix(y_test,Y_pred)
confusion_matrix

array([[34,  0,  0,  0],
       [ 1, 30,  3,  0],
       [ 0,  4, 39,  0],
       [ 0,  1,  3, 39]], dtype=int64)

**Random Forest**

In [21]:
parameters = { "max_features": ["sqrt","log2",None],
"min_samples_split": range(2,31,2) }
model = ensemble.RandomForestClassifier()        
grid=model_selection.GridSearchCV(model,parameters)
grid.fit(X_train_transformed,y_train.values.ravel())

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_features': ['sqrt', 'log2', None],
                         'min_samples_split': range(2, 31, 2)})

In [22]:
grid.best_params_

{'max_features': 'sqrt', 'min_samples_split': 6}

In [23]:
Y_pred = grid.predict( X_test_transformed )

In [24]:
model_forest=grid.best_estimator_
model_forest.score(X_test_transformed,y_test)

0.935064935064935

In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix=confusion_matrix(y_test,Y_pred)
confusion_matrix

array([[34,  0,  0,  0],
       [ 0, 31,  3,  0],
       [ 0,  3, 39,  1],
       [ 0,  0,  3, 40]], dtype=int64)